# Google Drive-லிருந்து .zip file-ஐ எடுத்துட்டு,
# அதை /content/extracted_zip folder-க்கு unzip பண்ணும் code இது.

In [1]:
import zipfile

zip_path = '/content/drive/MyDrive/Tokenizer_cholladi_folder/cholloadai-2021.txt.zip'
extract_path = '/content/extracted_zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipped from Drive.")


Unzipped from Drive.


# /content/extracted_zip folder-ல இருக்கும் .gz files-ஐ எல்லாம்  combined_output.txt என்ற single file-ஆ convert பண்ணும் Python code இது.


In [2]:
import gzip
import os

folder_path = "/content/extracted_zip"  # change if different
output_file_path = "/content/combined_output.txt"  # final combined file

# Open the output file in write mode
with open(output_file_path, 'wb') as outfile:
    # Iterate over all .gz files in the folder
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".gz"):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing: {filename}")
            # Open each .gz file and write its content to the output file
            with gzip.open(file_path, 'rb') as infile:
                outfile.write(infile.read())

print(" All .gz files have been concatenated into one file.")


Processing: cholloadai-2021.txt.00.gz
Processing: cholloadai-2021.txt.01.gz
Processing: cholloadai-2021.txt.02.gz
Processing: cholloadai-2021.txt.03.gz
Processing: cholloadai-2021.txt.04.gz
Processing: cholloadai-2021.txt.05.gz
Processing: cholloadai-2021.txt.06.gz
Processing: cholloadai-2021.txt.07.gz
Processing: cholloadai-2021.txt.08.gz
Processing: cholloadai-2021.txt.09.gz
Processing: cholloadai-2021.txt.10.gz
Processing: cholloadai-2021.txt.11.gz
Processing: cholloadai-2021.txt.12.gz
Processing: cholloadai-2021.txt.13.gz
Processing: cholloadai-2021.txt.14.gz
Processing: cholloadai-2021.txt.15.gz
Processing: cholloadai-2021.txt.16.gz
Processing: cholloadai-2021.txt.17.gz
Processing: cholloadai-2021.txt.18.gz
Processing: cholloadai-2021.txt.19.gz
Processing: cholloadai-2021.txt.20.gz
Processing: cholloadai-2021.txt.21.gz
Processing: cholloadai-2021.txt.22.gz
Processing: cholloadai-2021.txt.23.gz
Processing: cholloadai-2021.txt.24.gz
Processing: cholloadai-2021.txt.25.gz
Processing: 

# **முயற்சி 1:**

# combined_output.txt file-ஐ base-ஆ கொண்டு,
# Byte-level BPE tokenizer-ஐ train பண்ணி,
#.tokenizer_gpt folder-க்கு vocabulary (vocab.json) மற்றும் merge rules (merges.txt) files-ஆ save பண்ணும் process.

In [ ]:
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=["/content/combined_output.txt"], vocab_size=30000, min_frequency=2)
os.makedirs('.tokenizer_gpt',exist_ok=True)
tokenizer.save_model(".tokenizer_gpt")
tokenizer.save(".tokenizer_gpt/vocab.json", pretty=True)
tokenizer.save(".tokenizer_gpt/merges.txt")

# .tokenizer_gpt/vocab.json file-இருந்து tokenizer-ஐ load பண்ணி,
#[PAD]ன்னு ஒரு special padding token-ஐ add பண்ணும் Python code இது.

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer_file_path = os.path.abspath(".tokenizer_gpt/vocab.json")
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file=tokenizer_file_path
)
tokenizer.add_special_tokens({"pad_token": "[PAD]"})

1

In [ ]:
tokenizer.tokenize('இப்டியே பேசிக்கிட்டே இருந்தா எப்டி யாரு பெருசுனு அடிசிக்காட்டு ')

['à®ĩà®ª',
 'à¯į',
 'à®Ł',
 'à®¿',
 'à®¯',
 'à¯ĩ',
 'Ġà®ª',
 'à¯ĩ',
 'à®ļ',
 'à®¿',
 'à®ķ',
 'à¯į',
 'à®ķ',
 'à®¿',
 'à®Ł',
 'à¯į',
 'à®Ł',
 'à¯ĩ',
 'Ġà®ĩà®°',
 'à¯ģ',
 'à®¨',
 'à¯į',
 'à®¤',
 'à®¾',
 'Ġà®İà®ª',
 'à¯į',
 'à®Ł',
 'à®¿',
 'Ġà®¯',
 'à®¾',
 'à®°',
 'à¯ģ',
 'Ġà®ª',
 'à¯Ĩ',
 'à®°',
 'à¯ģ',
 'à®ļ',
 'à¯ģ',
 'à®©',
 'à¯ģ',
 'Ġà®ħà®Ł',
 'à®¿',
 'à®ļ',
 'à®¿',
 'à®ķ',
 'à¯į',
 'à®ķ',
 'à®¾',
 'à®Ł',
 'à¯į',
 'à®Ł',
 'à¯ģ',
 'Ġ']

In [ ]:
from google.colab import files
files.download('.tokenizer_gpt/vocab.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **முயற்சி 2:**

#BPE tokenizer creation,
#Tamil text (combined_output.txt)ல் train பண்ணி,
#Normalization, pre-tokenization, special tokens எல்லாம் configure பண்ணி,
#Final output-ஆ .json format-ல் saved.

In [ ]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, normalizers

tokenizer = Tokenizer(models.BPE())
tokenizer.normalizer = normalizers.Sequence([
    normalizers.NFKC(),
])
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

trainer = trainers.BpeTrainer(

    vocab_size=30000,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
)

tokenizer.train(["/content/combined_output.txt"], trainer)
tokenizer.save("tamil_tokenizer.json")


In [ ]:
import json

with open("vocab.json", "r", encoding="utf-8") as f:
    vocab = json.load(f)

# Print first 20 vocab tokens
for i, (token, idx) in enumerate(vocab.items()):
    print(f"{idx}: {token}")
    if i > 20:
        break


# **SentencePieace-Tokenizer**

In [ ]:
!pip install sentencepiece


In [ ]:
import sentencepiece as spm

spm.SentencePieceTrainer.Train(
    input='/content/combined_output.txt',
    model_prefix='tamil_unigram',
    vocab_size=15000,
    model_type='unigram',
    character_coverage=1.0,
    pad_id=0, unk_id=1, bos_id=2, eos_id=3,
    user_defined_symbols=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
)


In [ ]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.load("tamil_unigram.model")

# Tokenize a sentence
tokens = sp.encode("பெண்ணை பற்றி அவமதித்தார்கள்", out_type=str)
print("Tokens:", tokens)

# Decode back to sentence
decoded = sp.decode(tokens)
print("Decoded:", decoded)
